## __Check first before starting__

In [1]:
import os

# Change the working directory to the project root
# Working_directory = os.path.normpath("C:/Users/james/OneDrive/文件/Continual_Learning")
Working_directory = os.path.normpath("/mnt/mydisk/Continual_Learning_JL/Continual_Learning/")
os.chdir(Working_directory)
print(f"Working directory: {os.getcwd()}")

Working directory: /mnt/mydisk/Continual_Learning_JL/Continual_Learning


## __All imports__

In [2]:
# Operating system and file management
import os
import shutil
import contextlib
import traceback
import gc
import glob, copy
from collections import defaultdict
import subprocess
import time
import re

# Jupyter notebook widgets and display
import ipywidgets as widgets
from IPython.display import display

# Data manipulation and analysis
import pandas as pd
import numpy as np

# Plotting and visualization
import matplotlib.pyplot as plt
from mpl_interactions import zoom_factory, panhandler

# Machine learning and preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import pickle
from ta import trend, momentum, volatility, volume

# Mathematical and scientific computing
import math
from scipy.ndimage import gaussian_filter1d

# Type hinting
from typing import Callable, Tuple

# Deep learning with PyTorch
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset

## __Dataset Setup & Preprocessing__

### 📥 Step 1 — Set dataset path and load features & labels
We will load the 561-dimensional features and original activity labels for both training and testing.

In [3]:
# Set working directory manually before running this cell if needed
BASE_DIR = "Class_Incremental_CL/HAR_CIL/har_dataset"
TRAIN_DIR = os.path.join(BASE_DIR, "train")
TEST_DIR = os.path.join(BASE_DIR, "test")

# 活動對應（原始標籤）
activity_label_map = {
    1: "WALKING",
    2: "WALKING_UPSTAIRS",
    3: "WALKING_DOWNSTAIRS",
    4: "SITTING",
    5: "STANDING",
    6: "LAYING"
}

# Load feature and label data
X_train = np.loadtxt(os.path.join(TRAIN_DIR, "X_train.txt"))
y_train = np.loadtxt(os.path.join(TRAIN_DIR, "y_train.txt")).astype(int)
X_test = np.loadtxt(os.path.join(TEST_DIR, "X_test.txt"))
y_test = np.loadtxt(os.path.join(TEST_DIR, "y_test.txt")).astype(int)

print("Train shape:", X_train.shape, y_train.shape)
print("Test shape:", X_test.shape, y_test.shape)

Train shape: (7352, 561) (7352,)
Test shape: (2947, 561) (2947,)


### 🧩 Step 2 — Define period label groups and final class remapping
We will predefine the class mappings for each period to ensure consistency across the experiment.

In [4]:
# Period-to-original-label mapping
period_label_map = {
    1: [4, 5],           # SITTING, STANDING
    2: [4, 5, 1, 2, 3],  # + WALKING variants (merged)
    3: [4, 5, 1, 2, 3, 6],  # + LAYING
    4: [4, 5, 1, 2, 3, 6]   # same classes, but with walking variants separated
}

# Consistent label remapping across all periods (final label index)
final_class_map = {
    "SITTING": 0,
    "STANDING": 1,
    "WALKING": 2,
    "LAYING": 3,
    "WALKING_UPSTAIRS": 4,
    "WALKING_DOWNSTAIRS": 5
}

# Reverse mapping for readability
label_name_from_id = {v: k for k, v in final_class_map.items()}
activity_label_map = {
    1: "WALKING",
    2: "WALKING_UPSTAIRS",
    3: "WALKING_DOWNSTAIRS",
    4: "SITTING",
    5: "STANDING",
    6: "LAYING"
}


### 🧪 Step 3 — Filter and remap datasets for each period
We will build training and testing splits for each period using consistent class indices.

In [5]:
def map_label(label, period):
    name = activity_label_map[label]
    if period < 4:
        if name.startswith("WALKING"):
            return final_class_map["WALKING"]
        else:
            return final_class_map[name]
    else:
        return final_class_map[name]

def get_period_dataset(X, y, period):
    allowed_labels = period_label_map[period]
    mask = np.isin(y, allowed_labels)
    Xp = X[mask]
    yp = np.array([map_label(label, period) for label in y[mask]])
    return Xp, yp


### 📊 Step 4 — Print artistic class distribution for each period
We show each period's label map and class statistics to confirm correctness.

In [6]:
def print_class_distribution(y, var_name: str, label_map: dict) -> None:
    y = np.array(y).flatten()
    unique, counts = np.unique(y, return_counts=True)
    total = counts.sum()
    print(f"\n📦 Class Distribution for {var_name}")
    for i, c in zip(unique, counts):
        print(f"  ├─ Label {i:<2} ({label_map[i]:<20}) → {c:>5} samples ({(c/total)*100:>5.2f}%)")


In [7]:
period_datasets = {}

for period in range(1, 5):
    Xp_train, yp_train = get_period_dataset(X_train, y_train, period)
    Xp_test, yp_test = get_period_dataset(X_test, y_test, period)

    period_datasets[period] = {
        "train": (Xp_train, yp_train),
        "test": (Xp_test, yp_test)
    }

    print_class_distribution(yp_train, f"Period {period} (Train)", label_name_from_id)
    print_class_distribution(yp_test, f"Period {period} (Test)", label_name_from_id)



📦 Class Distribution for Period 1 (Train)
  ├─ Label 0  (SITTING             ) →  1286 samples (48.35%)
  ├─ Label 1  (STANDING            ) →  1374 samples (51.65%)

📦 Class Distribution for Period 1 (Test)
  ├─ Label 0  (SITTING             ) →   491 samples (48.00%)
  ├─ Label 1  (STANDING            ) →   532 samples (52.00%)

📦 Class Distribution for Period 2 (Train)
  ├─ Label 0  (SITTING             ) →  1286 samples (21.63%)
  ├─ Label 1  (STANDING            ) →  1374 samples (23.11%)
  ├─ Label 2  (WALKING             ) →  3285 samples (55.26%)

📦 Class Distribution for Period 2 (Test)
  ├─ Label 0  (SITTING             ) →   491 samples (20.37%)
  ├─ Label 1  (STANDING            ) →   532 samples (22.07%)
  ├─ Label 2  (WALKING             ) →  1387 samples (57.55%)

📦 Class Distribution for Period 3 (Train)
  ├─ Label 0  (SITTING             ) →  1286 samples (17.49%)
  ├─ Label 1  (STANDING            ) →  1374 samples (18.69%)
  ├─ Label 2  (WALKING             ) →  328

## __Check GPU, CUDA, Pytorch__

### GPU Details

In [8]:
!nvidia-smi

Wed Apr 23 13:54:47 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 570.86.15              Driver Version: 570.86.15      CUDA Version: 12.8     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA RTX A6000               Off |   00000000:2A:00.0 Off |                  Off |
| 54%   80C    P2            298W /  300W |   23769MiB /  49140MiB |    100%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

### CUDA Details

In [9]:
def check_gpu_config():
    """
    Check GPU availability and display detailed configuration information.
    """
    # Check if GPU is available
    gpu_available = torch.cuda.is_available()
    
    # Print header
    print("=" * 50)
    print("GPU Configuration Check".center(50))
    print("=" * 50)
    
    # Basic GPU availability
    print(f"{'PyTorch Version':<25}: {torch.__version__}")
    print(f"{'GPU Available':<25}: {'Yes' if gpu_available else 'No'}")
    
    # If GPU is available, print detailed info
    if gpu_available:
        print("-" * 50)
        print("GPU Details".center(50))
        print("-" * 50)
        
        # Device info
        print(f"{'Device Name':<25}: {torch.cuda.get_device_name(0)}")
        print(f"{'Number of GPUs':<25}: {torch.cuda.device_count()}")
        print(f"{'Current Device Index':<25}: {torch.cuda.current_device()}")
        
        # Compute capability and CUDA cores
        props = torch.cuda.get_device_properties(0)
        print(f"{'Compute Capability':<25}: {props.major}.{props.minor}")
        print(f"{'Total CUDA Cores':<25}: {props.multi_processor_count * 128}")  # Approx. 128 cores per SM
        
        # Memory info
        total_memory = props.total_memory / (1024 ** 3)  # Convert to GB
        memory_allocated = torch.cuda.memory_allocated(0) / (1024 ** 3)
        memory_reserved = torch.cuda.memory_reserved(0) / (1024 ** 3)
        print(f"{'Total Memory (GB)':<25}: {total_memory:.2f}")
        print(f"{'Allocated Memory (GB)':<25}: {memory_allocated:.2f}")
        print(f"{'Reserved Memory (GB)':<25}: {memory_reserved:.2f}")
    else:
        print("-" * 50)
        print("No GPU detected. Running on CPU.".center(50))
        print("-" * 50)
    
    print("=" * 50)

if __name__ == "__main__":
    check_gpu_config()

             GPU Configuration Check              
PyTorch Version          : 2.5.1
GPU Available            : Yes
--------------------------------------------------
                   GPU Details                    
--------------------------------------------------
Device Name              : NVIDIA RTX A6000
Number of GPUs           : 3
Current Device Index     : 0
Compute Capability       : 8.6
Total CUDA Cores         : 10752
Total Memory (GB)        : 47.41
Allocated Memory (GB)    : 0.00
Reserved Memory (GB)     : 0.00


### PyTorch Details

In [10]:
def print_torch_config():
    """Print PyTorch and CUDA configuration in a formatted manner."""
    print("=" * 50)
    print("PyTorch Configuration".center(50))
    print("=" * 50)
    
    # Basic PyTorch and CUDA info
    print(f"{'PyTorch Version':<25}: {torch.__version__}")
    print(f"{'CUDA Compiled Version':<25}: {torch.version.cuda}")
    print(f"{'CUDA Available':<25}: {'Yes' if torch.cuda.is_available() else 'No'}")
    print(f"{'Number of GPUs':<25}: {torch.cuda.device_count()}")

    # GPU details if available
    if torch.cuda.is_available():
        print(f"{'GPU Name':<25}: {torch.cuda.get_device_name(0)}")

    print("-" * 50)
    
    # Seed setting
    torch.manual_seed(42)
    print(f"{'Random Seed':<25}: 42 (Seeding successful!)")
    
    print("=" * 50)

if __name__ == "__main__":
    print_torch_config()

              PyTorch Configuration               
PyTorch Version          : 2.5.1
CUDA Compiled Version    : 12.1
CUDA Available           : Yes
Number of GPUs           : 3
GPU Name                 : NVIDIA RTX A6000
--------------------------------------------------
Random Seed              : 42 (Seeding successful!)


## __⚙️ GPU Selection — Auto-select the least loaded GPU__
This code automatically scans available GPUs and selects the one with the lowest current memory usage.


In [11]:
def auto_select_cuda_device(verbose=True):
    """
    Automatically selects the CUDA GPU with the least memory usage.
    Falls back to CPU if no GPU is available.
    """
    if not torch.cuda.is_available():
        print("🚫 No CUDA GPU available. Using CPU.")
        return torch.device("cpu")

    try:
        # Run nvidia-smi to get memory usage of each GPU
        smi_output = subprocess.check_output(
            ['nvidia-smi', '--query-gpu=memory.used', '--format=csv,nounits,noheader'],
            encoding='utf-8'
        )
        memory_used = [int(x) for x in smi_output.strip().split('\n')]
        best_gpu = int(np.argmin(memory_used))

        if verbose:
            print("🎯 Automatically selected GPU:")
            print(f"    - CUDA Device ID : {best_gpu}")
            print(f"    - Memory Used    : {memory_used[best_gpu]} MiB")
            print(f"    - Device Name    : {torch.cuda.get_device_name(best_gpu)}")
        return torch.device(f"cuda:{best_gpu}")
    except Exception as e:
        print(f"⚠️ Failed to auto-detect GPU. Falling back to cuda:0. ({e})")
        return torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Execute and assign
device = auto_select_cuda_device()

🎯 Automatically selected GPU:
    - CUDA Device ID : 1
    - Memory Used    : 18 MiB
    - Device Name    : NVIDIA RTX A6000


## __MLP Model__

### HAR_MLP_v1

In [12]:
class HAR_MLP_v1(nn.Module):
    def __init__(self, input_size: int, hidden_size: int, output_size: int, dropout: float = 0.2):
        super(HAR_MLP_v1, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(dropout)
        self.fc2 = nn.Linear(hidden_size, output_size)
        self.init_weights()

    def init_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight)
                nn.init.constant_(m.bias, 0)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc2(x)
        return x

### HAR_MLP_v2

In [ ]:
class HAR_MLP_v2(nn.Module):
    def __init__(self, input_size: int, hidden_size: int, output_size: int, dropout: float = 0.2):
        super(HAR_MLP_v2, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.bn1 = nn.BatchNorm1d(hidden_size)
        self.relu1 = nn.ReLU()
        self.dropout1 = nn.Dropout(dropout)

        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.bn2 = nn.BatchNorm1d(hidden_size)
        self.relu2 = nn.ReLU()
        self.dropout2 = nn.Dropout(dropout)

        self.fc3 = nn.Linear(hidden_size, output_size)
        self.init_weights()

    def init_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight)
                nn.init.constant_(m.bias, 0)

    def forward(self, x):
        x = self.fc1(x)
        x = self.bn1(x)
        x = self.relu1(x)
        x = self.dropout1(x)

        x = self.fc2(x)
        x = self.bn2(x)
        x = self.relu2(x)
        x = self.dropout2(x)

        x = self.fc3(x)
        return x


## __EWC Class Definition__

In [14]:
class EWC:
    def __init__(self, fisher: dict, params: dict):
        self.fisher = {k: v.cpu() for k, v in fisher.items()}
        self.params = {k: v.cpu() for k, v in params.items()}

    @staticmethod
    def compute_fisher_and_params(model, dataloader, criterion, device, sample_size=None):
        model.train()
        fisher = {n: torch.zeros_like(p, device=device) for n, p in model.named_parameters() if p.requires_grad}
        params = {n: p.clone().detach().cpu() for n, p in model.named_parameters() if p.requires_grad}

        total_samples = 0
        for i, (x, y) in enumerate(dataloader):
            if sample_size and total_samples >= sample_size:
                break
            x, y = x.to(device), y.to(device)
            model.zero_grad()
            outputs = model(x)
            # 可視模型輸出是否需展平而選擇加上 outputs.view(-1, ...)
            loss = criterion(outputs, y)
            loss.backward()
            for n, p in model.named_parameters():
                if p.grad is not None:
                    fisher[n] += (p.grad ** 2) * x.size(0)
            total_samples += x.size(0)

        fisher = {n: f / total_samples for n, f in fisher.items()}
        return {n: f.cpu() for n, f in fisher.items()}, params

    def penalty(self, model):
        loss = 0.0
        for n, p in model.named_parameters():
            if n in self.fisher:
                _loss = self.fisher[n].to(p.device) * (p - self.params[n].to(p.device)) ** 2
                loss += _loss.sum()
        return loss


## __Training Function with EWC__

In [15]:
def compute_classwise_accuracy(preds, targets, class_correct, class_total):
    """
    Computes per-class accuracy statistics from raw logits and ground truth.
    """
    preds = torch.argmax(preds, dim=-1)
    correct_mask = (preds == targets)
    for label in torch.unique(targets):
        label = label.item()
        label_mask = (targets == label)
        class_total[label] = class_total.get(label, 0) + label_mask.sum().item()
        class_correct[label] = class_correct.get(label, 0) + (correct_mask & label_mask).sum().item()

In [ ]:
def compute_fwt_har(previous_model, init_model, X_val, y_val, known_classes, batch_size=64):
    """
    FWT computation for HAR-style inputs (tabular MLP, not sequence).
    X_val: shape [N, F]
    y_val: shape [N]
    """
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    previous_model.to(device).eval()
    init_model.to(device).eval()

    # 只選擇 known_classes
    mask = np.isin(y_val, known_classes)
    X_known = X_val[mask]
    y_known = y_val[mask]

    if len(y_known) == 0:
        print(f"⚠️ No validation samples for known classes {known_classes}.")
        return None, None, None

    print(f"📋 Total samples for known classes {known_classes}: {len(y_known)}")

    dataset = TensorDataset(
        torch.tensor(X_known, dtype=torch.float32),
        torch.tensor(y_known, dtype=torch.long)
    )
    loader = DataLoader(dataset, batch_size=batch_size)

    correct_prev, correct_init, total = 0, 0, 0

    with torch.no_grad():
        for xb, yb in loader:
            xb, yb = xb.to(device), yb.to(device)

            out_prev = previous_model(xb)
            out_init = init_model(xb)

            preds_prev = torch.argmax(out_prev, dim=-1)
            preds_init = torch.argmax(out_init, dim=-1)

            correct_prev += (preds_prev == yb).sum().item()
            correct_init += (preds_init == yb).sum().item()
            total += yb.size(0)

    acc_prev = correct_prev / total
    acc_init = correct_init / total
    fwt_value = acc_prev - acc_init

    print(f"\n### 🔍 FWT Debug Info (HAR):")
    print(f"- Total evaluated samples: {total}")
    print(f"- Correct (PrevModel): {correct_prev} / {total} → Acc = {acc_prev:.4f}")
    print(f"- Correct (InitModel): {correct_init} / {total} → Acc = {acc_init:.4f}")
    print(f"- FWT = Acc_prev - Acc_init = {fwt_value:.4f}")

    return fwt_value, acc_prev, acc_init


In [ ]:
def print_model_info(model):
    total_params = sum(p.numel() for p in model.parameters())
    param_size_bytes = total_params * 4  # 假設 float32，每個參數佔 4 bytes
    param_size_MB = param_size_bytes / (1024**2)

    print(f"- Total Parameters: {total_params}")
    print(f"- Model Size (float32): {param_size_MB:.2f} MB")

In [16]:
def train_with_ewc(model, output_size, criterion, optimizer,
                   X_train, y_train, X_val, y_val,
                   scheduler=None, num_epochs=10, batch_size=64,
                   model_saving_folder=None, model_name=None,
                   stop_signal_file=None, ewc=None, lambda_ewc=0.4, device=auto_select_cuda_device()):

    print("\n🚀 'train_with_ewc' started.")
    model_name = model_name or 'model'
    model_saving_folder = model_saving_folder or './saved_models'
    
    if model_saving_folder:
        if os.path.exists(model_saving_folder):
            shutil.rmtree(model_saving_folder)
            print(f"✅ Removed existing folder: {model_saving_folder}")
        os.makedirs(model_saving_folder, exist_ok=True)

    device = device
    model.to(device)

    X_train = torch.tensor(X_train, dtype=torch.float32).to(device)
    y_train = torch.tensor(y_train, dtype=torch.long).to(device)
    X_val = torch.tensor(X_val, dtype=torch.float32).to(device)
    y_val = torch.tensor(y_val, dtype=torch.long).to(device)

    train_loader = DataLoader(TensorDataset(X_train, y_train), batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(TensorDataset(X_val, y_val), batch_size=batch_size, shuffle=False)

    print("\n✅ Data Overview:")
    print(f"X_train: {X_train.shape}, y_train: {y_train.shape}")
    print(f"X_val: {X_val.shape}, y_val: {y_val.shape}")

    # Start timer
    start_time = time.time()

    best_results = []
    for epoch in range(num_epochs):
        if stop_signal_file and os.path.exists(stop_signal_file):
            print("\n🛑 Stop signal detected. Exiting training loop.")
            break

        model.train()
        epoch_loss = 0.0
        class_correct, class_total = {}, {}

        for X_batch, y_batch in train_loader:
            optimizer.zero_grad()
            outputs = model(X_batch).view(-1, output_size)
            y_batch = y_batch.view(-1)
            loss = criterion(outputs, y_batch)
            if ewc is not None:
                loss += (lambda_ewc / 2) * ewc.penalty(model)
            loss.backward()
            optimizer.step()

            epoch_loss += loss.item() * X_batch.size(0)
            compute_classwise_accuracy(outputs, y_batch, class_correct, class_total)

        train_loss = epoch_loss / len(train_loader.dataset)
        train_acc = {int(c): f"{(class_correct[c] / class_total[c]) * 100:.2f}%" if class_total[c] > 0 else "0.00%" for c in sorted(class_total.keys())}

        model.eval()
        val_loss, val_correct, val_total = 0.0, 0, 0
        val_class_correct, val_class_total = {}, {}
        with torch.no_grad():
            for X_batch, y_batch in val_loader:
                outputs = model(X_batch).view(-1, output_size)
                y_batch = y_batch.view(-1)
                val_loss += criterion(outputs, y_batch).item() * X_batch.size(0)
                predictions = torch.argmax(outputs, dim=-1)
                val_correct += (predictions == y_batch).sum().item()
                val_total += y_batch.size(0)
                compute_classwise_accuracy(outputs, y_batch, val_class_correct, val_class_total)

        val_loss /= len(val_loader.dataset)
        val_acc = val_correct / val_total
        val_acc_cls = {int(c): f"{(val_class_correct[c] / val_class_total[c]) * 100:.2f}%" if val_class_total[c] > 0 else "0.00%" for c in sorted(val_class_total.keys())}

        print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.6f}, Train-Class-Acc: {train_acc},")
        print(f"Val Loss: {val_loss:.6f}, Val Acc: {val_acc * 100:.2f}%, Val-Class-Acc: {val_acc_cls}, LR: {optimizer.param_groups[0]['lr']:.6f}")

        model_path = os.path.join(model_saving_folder, f"{model_name}_epoch_{epoch+1}.pth")
        current = {
            'epoch': epoch + 1,
            'train_loss': train_loss,
            'val_loss': val_loss,
            'val_accuracy': val_acc,
            'train_classwise_accuracy': train_acc,
            'val_classwise_accuracy': val_acc_cls,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'learning_rate': optimizer.param_groups[0]['lr'],
            'model_path': model_path
        }

        if len(best_results) < 5 or val_acc > best_results[-1]['val_accuracy']:
            if len(best_results) == 5:
                to_remove = best_results.pop()
                if os.path.exists(to_remove['model_path']):
                    os.remove(to_remove['model_path'])
                    print(f"🗑 Removed: {to_remove['model_path']}")
            best_results.append(current)
            best_results.sort(key=lambda x: (x['val_accuracy'], x['epoch']), reverse=True)
            torch.save(current, model_path)
            print(f"✅ Saved model: {model_path}")

        if scheduler: scheduler.step(val_loss)

    # End timer and report
    elapsed_time = time.time() - start_time
    print(f"\n⏳ Total training time: {elapsed_time:.2f} seconds")
    
    if best_results:
        best = best_results[0]
        best_model_path = os.path.join(model_saving_folder, f"{model_name}_best.pth")
        torch.save(best, best_model_path)
        print(f"\n🏆 Best model saved as: {best_model_path} (Val Accuracy: {best['val_accuracy'] * 100:.2f}%)")

    final_model_path = os.path.join(model_saving_folder, f"{model_name}_final.pth")
    torch.save(current, final_model_path)
    print(f"\n📌 Final model saved as: {final_model_path}")

    print("\n🎯 Top 5 Best Models:")
    for res in best_results:
        print(f"Epoch {res['epoch']}, Train Loss: {res['train_loss']:.6f}, Train-Acc: {res['train_classwise_accuracy']},\n"
              f"Val Loss: {res['val_loss']:.6f}, Val Acc: {res['val_accuracy']*100:.2f}%, Val-Acc: {res['val_classwise_accuracy']},"
              f" Model Path: {res['model_path']}")

    # === Markdown summary for best result ===
    best_model = max(best_results, key=lambda x: x['val_accuracy'])

    # Extract period number from folder path
    match = re.search(r'Period_(\d+)', model_saving_folder)
    period_label = match.group(1) if match else "?"
    model_name_str = model.__class__.__name__

    best_md_summary = f"""
    ---
### Period {period_label}
+ ##### Total training time: {elapsed_time:.2f} seconds
+ ##### Model: {model_name_str}
+ ##### Training and saving in *'{model_saving_folder}'*
+ ##### Best Epoch: {best_model['epoch']}
#### __Val Accuracy: {best_model['val_accuracy'] * 100:.2f}%__
#### __Val-Class-Acc: {best_model['val_classwise_accuracy']}__
    """
    print(best_md_summary.strip())

    del X_train, y_train, X_val, y_val, train_loader, val_loader
    torch.cuda.empty_cache()
    gc.collect()


🎯 Automatically selected GPU:
    - CUDA Device ID : 1
    - Memory Used    : 18 MiB
    - Device Name    : NVIDIA RTX A6000


## __📋 Label Mapping (HAR Activities)__

| Label ID | Activity Name       |
|----------|---------------------|
|    0     | SITTING             |
|    1     | STANDING            |
|    2     | WALKING             |
|    3     | LAYING              |
|    4     | WALKING_UPSTAIRS    |
|    5     | WALKING_DOWNSTAIRS  |

## __Common Parameters__

In [ ]:
batch_size    = 64
stop_signal_file = os.path.normpath(os.path.join('Class_Incremental_CL', 'HAR_CIL/stop_training.txt'))

## __HAR_MLP_v1 Training__

---
### Period 1
+ ##### Total training time: 79.13 seconds
+ ##### Model: HAR_MLP_v1
+ ##### Training and saving in *'Class_Incremental_CL/HAR_CIL/Trained_models/EWC_CIL_v1/Period_1'*
+ ##### Best Epoch: 965
#### __Val Accuracy: 94.62%__
#### __Val-Class-Acc: {0: '91.04%', 1: '97.93%'}__

In [ ]:
# ================================
# 📌 Period 1 Configuration
# ================================
period = 1

# Load processed dataset for this period
X_train, y_train = period_datasets[period]["train"]
X_val, y_val     = period_datasets[period]["test"]   # use test set for validation

# Auto-select the best CUDA device
device = auto_select_cuda_device()

# Model and training hyperparameters
input_size    = X_train.shape[1]
hidden_size   = 128
output_size   = len(set(y_train))
dropout       = 0.2
learning_rate = 0.0001
weight_decay  = 1e-5
num_epochs    = 1000
lambda_ewc    = 0.0  # EWC disabled for Period 1
ewc_state     = None

# Model saving path
model_name = "HAR_MLP"
model_saving_folder = os.path.join("Class_Incremental_CL", "HAR_CIL", "Trained_models", "EWC_CIL_v1", "Period_1")
os.makedirs(model_saving_folder, exist_ok=True)

# Initialize model, criterion, optimizer
model     = HAR_MLP_v1(input_size=input_size, hidden_size=hidden_size, output_size=output_size, dropout=dropout).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)

# Train the model
train_with_ewc(
    model=model,
    output_size=output_size,
    criterion=criterion,
    optimizer=optimizer,
    X_train=X_train,
    y_train=y_train,
    X_val=X_val,
    y_val=y_val,
    scheduler=None,
    num_epochs=num_epochs,
    batch_size=batch_size,
    model_saving_folder=model_saving_folder,
    model_name=model_name,
    stop_signal_file=stop_signal_file,
    ewc=ewc_state,
    lambda_ewc=lambda_ewc,
    device=device
)

# ================================
# ✅ Summary & Resource Cleanup
# ================================

# Unique class summary
unique_classes = np.unique(y_train)
num_classes = len(unique_classes)

print(f"\n✅ Training Complete. Final model architecture:")
print(model)
print(f"📊 Class Summary → unique_classes = {unique_classes}, num_classes = {num_classes}")

# Clean up memory to prepare for next period
del X_train, y_train, X_val, y_val, model
gc.collect()
torch.cuda.empty_cache()


🎯 Automatically selected GPU:
    - CUDA Device ID : 1
    - Memory Used    : 11269 MiB
    - Device Name    : NVIDIA RTX A6000

🚀 'train_with_ewc' started.
✅ Removed existing folder: Class_Incremental_CL/HAR_CIL/Trained_models/EWC_CIL_v1/Period_1

✅ Data Overview:
X_train: torch.Size([2660, 561]), y_train: torch.Size([2660])
X_val: torch.Size([1023, 561]), y_val: torch.Size([1023])
Epoch 1/1000, Train Loss: 0.735055, Train-Class-Acc: {0: '53.34%', 1: '59.24%'},
Val Loss: 0.566365, Val Acc: 80.84%, Val-Class-Acc: {0: '87.78%', 1: '74.44%'}, LR: 0.000100
✅ Saved model: Class_Incremental_CL/HAR_CIL/Trained_models/EWC_CIL_v1/Period_1/HAR_MLP_epoch_1.pth
Epoch 2/1000, Train Loss: 0.585920, Train-Class-Acc: {0: '66.10%', 1: '71.91%'},
Val Loss: 0.511184, Val Acc: 77.13%, Val-Class-Acc: {0: '56.01%', 1: '96.62%'}, LR: 0.000100
✅ Saved model: Class_Incremental_CL/HAR_CIL/Trained_models/EWC_CIL_v1/Period_1/HAR_MLP_epoch_2.pth
Epoch 3/1000, Train Loss: 0.521991, Train-Class-Acc: {0: '71.93%', 1

---
### Period 2
+ ##### Total training time: 243.23 seconds
+ ##### Model: HAR_MLP_v1
+ ##### Training and saving in *'Class_Incremental_CL/HAR_CIL/Trained_models/EWC_CIL_v1/Period_2'*
+ ##### Best Epoch: 935
#### __Val Accuracy: 97.72%__
#### __Val-Class-Acc: {0: '90.22%', 1: '98.68%', 2: '100.00%'}__

In [ ]:
# ================================
# 📌 Period 2 Configuration
# ================================
period = 2

# Load dataset
X_train, y_train = period_datasets[period]["train"]
X_val, y_val     = period_datasets[period]["test"]

# Auto-select device
device = auto_select_cuda_device()

# Set model and training config
input_size    = X_train.shape[1]
hidden_size   = 128
output_size   = len(set(y_train))
dropout       = 0.2
learning_rate = 0.0001
weight_decay  = 1e-5
num_epochs    = 1000
lambda_ewc    = 1.0

# Model saving directory
model_name = "HAR_MLP"
model_saving_folder = os.path.join("Class_Incremental_CL", "HAR_CIL", "Trained_models", "EWC_CIL_v1", "Period_2")
os.makedirs(model_saving_folder, exist_ok=True)

# Load best model from Period 1 (ignore output layer shape mismatch)
prev_path = os.path.join("Class_Incremental_CL", "HAR_CIL", "Trained_models", "EWC_CIL_v1", "Period_1", f"{model_name}_best.pth")
prev_checkpoint = torch.load(prev_path, map_location=device)
model = HAR_MLP_v1(input_size, hidden_size, output_size, dropout).to(device)

# Filter mismatched keys (especially fc2) during load
state_dict = prev_checkpoint["model_state_dict"]
model_dict = model.state_dict()
filtered_dict = {k: v for k, v in state_dict.items() if k in model_dict and model_dict[k].shape == v.shape}
model.load_state_dict(filtered_dict, strict=False)

# Compute Fisher Information Matrix (based on Period 1 training data)
criterion = nn.CrossEntropyLoss()
train_loader = DataLoader(
    TensorDataset(
        torch.tensor(period_datasets[1]["train"][0], dtype=torch.float32),
        torch.tensor(period_datasets[1]["train"][1], dtype=torch.long)
    ),
    batch_size=batch_size, shuffle=True
)
fisher_dict, params_dict = EWC.compute_fisher_and_params(model, train_loader, criterion, device=device)
ewc_state = EWC(fisher_dict, params_dict)

# Optimizer (new for Period 2)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)

# Train with EWC
train_with_ewc(
    model=model,
    output_size=output_size,
    criterion=criterion,
    optimizer=optimizer,
    X_train=X_train,
    y_train=y_train,
    X_val=X_val,
    y_val=y_val,
    scheduler=None,
    num_epochs=num_epochs,
    batch_size=batch_size,
    model_saving_folder=model_saving_folder,
    model_name=model_name,
    stop_signal_file=stop_signal_file,
    ewc=ewc_state,
    lambda_ewc=lambda_ewc,
    device=device
)

# ================================
# ✅ Summary & Resource Cleanup
# ================================
unique_classes = np.unique(y_train)
num_classes = len(unique_classes)

print(f"\n✅ Training Complete. Final model architecture:")
print(model)
print(f"📊 Class Summary → unique_classes = {unique_classes}, num_classes = {num_classes}")

# Memory cleanup
del X_train, y_train, X_val, y_val, train_loader, prev_checkpoint, state_dict, model_dict, filtered_dict
del fisher_dict, params_dict, ewc_state, model
gc.collect()
torch.cuda.empty_cache()


🎯 Automatically selected GPU:
    - CUDA Device ID : 0
    - Memory Used    : 27486 MiB
    - Device Name    : NVIDIA RTX A6000

🚀 'train_with_ewc' started.
✅ Removed existing folder: Class_Incremental_CL/HAR_CIL/Trained_models/EWC_CIL_v1/Period_2

✅ Data Overview:
X_train: torch.Size([5945, 561]), y_train: torch.Size([5945])
X_val: torch.Size([2410, 561]), y_val: torch.Size([2410])


/tmp/ipykernel_3491776/1503672125.py:31: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  prev_checkpoint = torch.load(prev_path, map_location=device)


Epoch 1/1000, Train Loss: 0.403891, Train-Class-Acc: {0: '82.35%', 1: '72.42%', 2: '96.23%'},
Val Loss: 0.166479, Val Acc: 96.60%, Val-Class-Acc: {0: '85.13%', 1: '98.31%', 2: '100.00%'}, LR: 0.000100
✅ Saved model: Class_Incremental_CL/HAR_CIL/Trained_models/EWC_CIL_v1/Period_2/HAR_MLP_epoch_1.pth
Epoch 2/1000, Train Loss: 0.144973, Train-Class-Acc: {0: '90.44%', 1: '93.96%', 2: '99.97%'},
Val Loss: 0.125727, Val Acc: 96.97%, Val-Class-Acc: {0: '87.37%', 1: '97.93%', 2: '100.00%'}, LR: 0.000100
✅ Saved model: Class_Incremental_CL/HAR_CIL/Trained_models/EWC_CIL_v1/Period_2/HAR_MLP_epoch_2.pth
Epoch 3/1000, Train Loss: 0.109035, Train-Class-Acc: {0: '93.78%', 1: '95.34%', 2: '99.97%'},
Val Loss: 0.112250, Val Acc: 97.10%, Val-Class-Acc: {0: '87.78%', 1: '98.12%', 2: '100.00%'}, LR: 0.000100
✅ Saved model: Class_Incremental_CL/HAR_CIL/Trained_models/EWC_CIL_v1/Period_2/HAR_MLP_epoch_3.pth
Epoch 4/1000, Train Loss: 0.094232, Train-Class-Acc: {0: '94.87%', 1: '95.92%', 2: '99.94%'},
Val Lo

---
### Period 3
+ ##### Total training time: 263.91 seconds
+ ##### Model: HAR_MLP_v1
+ ##### Training and saving in *'Class_Incremental_CL/HAR_CIL/Trained_models/EWC_CIL_v1/Period_3'*
+ ##### Best Epoch: 871
#### __Val Accuracy: 98.07%__
#### __Val-Class-Acc: {0: '90.63%', 1: '97.93%', 2: '100.00%', 3: '100.00%'}__

In [ ]:
# ================================
# 📌 Period 3 Configuration
# ================================
period = 3

# Load dataset
X_train, y_train = period_datasets[period]["train"]
X_val, y_val     = period_datasets[period]["test"]

# Auto-select device
device = auto_select_cuda_device()

# Set model and training config
input_size    = X_train.shape[1]
hidden_size   = 128
output_size   = len(set(y_train))
dropout       = 0.2
learning_rate = 0.0001
weight_decay  = 1e-5
num_epochs    = 1000
lambda_ewc    = 1.0

# Model saving directory
model_name = "HAR_MLP"
model_saving_folder = os.path.join("Class_Incremental_CL", "HAR_CIL", "Trained_models", "EWC_CIL_v1", "Period_3")
os.makedirs(model_saving_folder, exist_ok=True)

# Load best model from Period 2 (ignore output layer shape mismatch)
prev_path = os.path.join("Class_Incremental_CL", "HAR_CIL", "Trained_models", "EWC_CIL_v1", "Period_2", f"{model_name}_best.pth")
prev_checkpoint = torch.load(prev_path, map_location=device)
model = HAR_MLP_v1(input_size, hidden_size, output_size, dropout).to(device)

# Filter mismatched keys (especially fc2) during load
state_dict = prev_checkpoint["model_state_dict"]
model_dict = model.state_dict()
filtered_dict = {k: v for k, v in state_dict.items() if k in model_dict and model_dict[k].shape == v.shape}
model.load_state_dict(filtered_dict, strict=False)

# Compute Fisher from Period 2 training data
criterion = nn.CrossEntropyLoss()
train_loader = DataLoader(
    TensorDataset(
        torch.tensor(period_datasets[2]["train"][0], dtype=torch.float32),
        torch.tensor(period_datasets[2]["train"][1], dtype=torch.long)
    ),
    batch_size=batch_size, shuffle=True
)
fisher_dict, params_dict = EWC.compute_fisher_and_params(model, train_loader, criterion, device=device)
ewc_state = EWC(fisher_dict, params_dict)

# Optimizer for this period
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)

# Train with EWC
train_with_ewc(
    model=model,
    output_size=output_size,
    criterion=criterion,
    optimizer=optimizer,
    X_train=X_train,
    y_train=y_train,
    X_val=X_val,
    y_val=y_val,
    scheduler=None,
    num_epochs=num_epochs,
    batch_size=batch_size,
    model_saving_folder=model_saving_folder,
    model_name=model_name,
    stop_signal_file=stop_signal_file,
    ewc=ewc_state,
    lambda_ewc=lambda_ewc,
    device=device
)

# ================================
# ✅ Summary & Resource Cleanup
# ================================
unique_classes = np.unique(y_train)
num_classes = len(unique_classes)

print(f"\n✅ Training Complete. Final model architecture:")
print(model)
print(f"📊 Class Summary → unique_classes = {unique_classes}, num_classes = {num_classes}")

del X_train, y_train, X_val, y_val, train_loader, prev_checkpoint, state_dict, model_dict, filtered_dict
del fisher_dict, params_dict, ewc_state, model
gc.collect()
torch.cuda.empty_cache()


🎯 Automatically selected GPU:
    - CUDA Device ID : 0
    - Memory Used    : 26557 MiB
    - Device Name    : NVIDIA RTX A6000

🚀 'train_with_ewc' started.
✅ Removed existing folder: Class_Incremental_CL/HAR_CIL/Trained_models/EWC_CIL_v1/Period_3

✅ Data Overview:
X_train: torch.Size([7352, 561]), y_train: torch.Size([7352])
X_val: torch.Size([2947, 561]), y_val: torch.Size([2947])


/tmp/ipykernel_3491776/1351216394.py:31: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  prev_checkpoint = torch.load(prev_path, map_location=device)


Epoch 1/1000, Train Loss: 0.825303, Train-Class-Acc: {0: '46.89%', 1: '55.46%', 2: '84.35%', 3: '72.00%'},
Val Loss: 0.244195, Val Acc: 94.06%, Val-Class-Acc: {0: '82.28%', 1: '95.30%', 2: '100.00%', 3: '88.27%'}, LR: 0.000100
✅ Saved model: Class_Incremental_CL/HAR_CIL/Trained_models/EWC_CIL_v1/Period_3/HAR_MLP_epoch_1.pth
Epoch 2/1000, Train Loss: 0.336030, Train-Class-Acc: {0: '65.09%', 1: '82.53%', 2: '99.57%', 3: '85.50%'},
Val Loss: 0.188144, Val Acc: 94.84%, Val-Class-Acc: {0: '84.93%', 1: '96.80%', 2: '100.00%', 3: '88.64%'}, LR: 0.000100
✅ Saved model: Class_Incremental_CL/HAR_CIL/Trained_models/EWC_CIL_v1/Period_3/HAR_MLP_epoch_2.pth
Epoch 3/1000, Train Loss: 0.215803, Train-Class-Acc: {0: '79.63%', 1: '88.50%', 2: '99.88%', 3: '90.97%'},
Val Loss: 0.154940, Val Acc: 95.45%, Val-Class-Acc: {0: '87.37%', 1: '96.43%', 2: '100.00%', 3: '90.13%'}, LR: 0.000100
✅ Saved model: Class_Incremental_CL/HAR_CIL/Trained_models/EWC_CIL_v1/Period_3/HAR_MLP_epoch_3.pth
Epoch 4/1000, Train Lo

---
### Period 4
+ ##### Total training time: 272.42 seconds
+ ##### Model: HAR_MLP_v1
+ ##### Training and saving in *'Class_Incremental_CL/HAR_CIL/Trained_models/EWC_CIL_v1/Period_4'*
+ ##### Best Epoch: 841
#### __Val Accuracy: 96.27%__
#### __Val-Class-Acc: {0: '90.63%', 1: '97.56%', 2: '97.18%', 3: '99.81%', 4: '98.30%', 5: '93.33%'}__

In [ ]:
# ================================
# 📌 Period 4 Configuration
# ================================
period = 4

# Load dataset
X_train, y_train = period_datasets[period]["train"]
X_val, y_val     = period_datasets[period]["test"]

# Auto-select device
device = auto_select_cuda_device()

# Set model and training config
input_size    = X_train.shape[1]
hidden_size   = 128
output_size   = len(set(y_train))
dropout       = 0.2
learning_rate = 0.0001
weight_decay  = 1e-5
num_epochs    = 1000
lambda_ewc    = 1.0

# Model saving directory
model_name = "HAR_MLP"
model_saving_folder = os.path.join("Class_Incremental_CL", "HAR_CIL", "Trained_models", "EWC_CIL_v1", "Period_4")
os.makedirs(model_saving_folder, exist_ok=True)

# Load best model from Period 3 (ignore output layer shape mismatch)
prev_path = os.path.join("Class_Incremental_CL", "HAR_CIL", "Trained_models", "EWC_CIL_v1", "Period_3", f"{model_name}_best.pth")
prev_checkpoint = torch.load(prev_path, map_location=device)
model = HAR_MLP_v1(input_size, hidden_size, output_size, dropout).to(device)

# Filter mismatched keys (especially fc2) during load
state_dict = prev_checkpoint["model_state_dict"]
model_dict = model.state_dict()
filtered_dict = {k: v for k, v in state_dict.items() if k in model_dict and model_dict[k].shape == v.shape}
model.load_state_dict(filtered_dict, strict=False)

# Compute Fisher from Period 3 training data
criterion = nn.CrossEntropyLoss()
train_loader = DataLoader(
    TensorDataset(
        torch.tensor(period_datasets[3]["train"][0], dtype=torch.float32),
        torch.tensor(period_datasets[3]["train"][1], dtype=torch.long)
    ),
    batch_size=batch_size, shuffle=True
)
fisher_dict, params_dict = EWC.compute_fisher_and_params(model, train_loader, criterion, device=device)
ewc_state = EWC(fisher_dict, params_dict)

# Optimizer for this period
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)

# Train with EWC
train_with_ewc(
    model=model,
    output_size=output_size,
    criterion=criterion,
    optimizer=optimizer,
    X_train=X_train,
    y_train=y_train,
    X_val=X_val,
    y_val=y_val,
    scheduler=None,
    num_epochs=num_epochs,
    batch_size=batch_size,
    model_saving_folder=model_saving_folder,
    model_name=model_name,
    stop_signal_file=stop_signal_file,
    ewc=ewc_state,
    lambda_ewc=lambda_ewc,
    device=device
)

# ================================
# ✅ Summary & Resource Cleanup
# ================================
unique_classes = np.unique(y_train)
num_classes = len(unique_classes)

print(f"\n✅ Training Complete. Final model architecture:")
print(model)
print(f"📊 Class Summary → unique_classes = {unique_classes}, num_classes = {num_classes}")

del X_train, y_train, X_val, y_val, train_loader, prev_checkpoint, state_dict, model_dict, filtered_dict
del fisher_dict, params_dict, ewc_state, model
gc.collect()
torch.cuda.empty_cache()


🎯 Automatically selected GPU:
    - CUDA Device ID : 0
    - Memory Used    : 26557 MiB
    - Device Name    : NVIDIA RTX A6000

🚀 'train_with_ewc' started.
✅ Removed existing folder: Class_Incremental_CL/HAR_CIL/Trained_models/EWC_CIL_v1/Period_4

✅ Data Overview:
X_train: torch.Size([7352, 561]), y_train: torch.Size([7352])
X_val: torch.Size([2947, 561]), y_val: torch.Size([2947])


/tmp/ipykernel_3491776/3642449888.py:31: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  prev_checkpoint = torch.load(prev_path, map_location=device)


Epoch 1/1000, Train Loss: 1.216455, Train-Class-Acc: {0: '51.09%', 1: '63.76%', 2: '43.39%', 3: '65.17%', 4: '48.65%', 5: '44.02%'},
Val Loss: 0.638175, Val Acc: 79.95%, Val-Class-Acc: {0: '80.45%', 1: '98.12%', 2: '64.92%', 3: '88.64%', 4: '70.28%', 5: '73.81%'}, LR: 0.000100
✅ Saved model: Class_Incremental_CL/HAR_CIL/Trained_models/EWC_CIL_v1/Period_4/HAR_MLP_epoch_1.pth
Epoch 2/1000, Train Loss: 0.593916, Train-Class-Acc: {0: '80.33%', 1: '88.86%', 2: '65.01%', 3: '90.69%', 4: '66.54%', 5: '67.34%'},
Val Loss: 0.483212, Val Acc: 88.26%, Val-Class-Acc: {0: '88.59%', 1: '96.05%', 2: '85.28%', 3: '89.57%', 4: '94.27%', 5: '73.10%'}, LR: 0.000100
✅ Saved model: Class_Incremental_CL/HAR_CIL/Trained_models/EWC_CIL_v1/Period_4/HAR_MLP_epoch_2.pth
Epoch 3/1000, Train Loss: 0.453118, Train-Class-Acc: {0: '86.55%', 1: '91.63%', 2: '78.06%', 3: '93.03%', 4: '76.98%', 5: '74.04%'},
Val Loss: 0.419803, Val Acc: 88.19%, Val-Class-Acc: {0: '88.59%', 1: '96.43%', 2: '93.35%', 3: '90.32%', 4: '82.3

## __HAR_MLP_v2 Training__

---
### Period 1
+ ##### Total training time: 86.52 seconds
+ ##### Model: HAR_MLP_v2
+ ##### Training and saving in *'Class_Incremental_CL/HAR_CIL/Trained_models/EWC_CIL_v2/Period_1'*
+ ##### Best Epoch: 351
#### __Val Accuracy: 95.21%__
#### __Val-Class-Acc: {0: '92.87%', 1: '97.37%'}__

In [ ]:
# ================================
# 📌 Period 1 Configuration
# ================================
period = 1

# Load processed dataset for this period
X_train, y_train = period_datasets[period]["train"]
X_val, y_val     = period_datasets[period]["test"]   # use test set for validation

# Auto-select the best CUDA device
device = auto_select_cuda_device()

# Model and training hyperparameters
input_size    = X_train.shape[1]
hidden_size   = 128
output_size   = len(set(y_train))
dropout       = 0.2
learning_rate = 0.0001
weight_decay  = 1e-5
num_epochs    = 1000
lambda_ewc    = 0.0  # EWC disabled for Period 1
ewc_state     = None

# Model saving path
model_name = "HAR_MLP"
model_saving_folder = os.path.join("Class_Incremental_CL", "HAR_CIL", "Trained_models", "EWC_CIL_v2", "Period_1")
os.makedirs(model_saving_folder, exist_ok=True)

# Initialize model, criterion, optimizer
model     = HAR_MLP_v2(input_size=input_size, hidden_size=hidden_size, output_size=output_size, dropout=dropout).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)

# Train the model
train_with_ewc(
    model=model,
    output_size=output_size,
    criterion=criterion,
    optimizer=optimizer,
    X_train=X_train,
    y_train=y_train,
    X_val=X_val,
    y_val=y_val,
    scheduler=None,
    num_epochs=num_epochs,
    batch_size=batch_size,
    model_saving_folder=model_saving_folder,
    model_name=model_name,
    stop_signal_file=stop_signal_file,
    ewc=ewc_state,
    lambda_ewc=lambda_ewc,
    device=device
)

# ================================
# ✅ Summary & Resource Cleanup
# ================================

# Unique class summary
unique_classes = np.unique(y_train)
num_classes = len(unique_classes)

print(f"\n✅ Training Complete. Final model architecture:")
print(model)
print(f"📊 Class Summary → unique_classes = {unique_classes}, num_classes = {num_classes}")

# Clean up memory to prepare for next period
del X_train, y_train, X_val, y_val, model
gc.collect()
torch.cuda.empty_cache()


🎯 Automatically selected GPU:
    - CUDA Device ID : 0
    - Memory Used    : 37218 MiB
    - Device Name    : NVIDIA RTX A6000

🚀 'train_with_ewc' started.
✅ Removed existing folder: Class_Incremental_CL/HAR_CIL/Trained_models/EWC_CIL_v2/Period_1

✅ Data Overview:
X_train: torch.Size([2660, 561]), y_train: torch.Size([2660])
X_val: torch.Size([1023, 561]), y_val: torch.Size([1023])
Epoch 1/1000, Train Loss: 0.779738, Train-Class-Acc: {0: '32.35%', 1: '85.37%'},
Val Loss: 0.823068, Val Acc: 55.52%, Val-Class-Acc: {0: '7.54%', 1: '99.81%'}, LR: 0.000100
✅ Saved model: Class_Incremental_CL/HAR_CIL/Trained_models/EWC_CIL_v2/Period_1/HAR_MLP_epoch_1.pth
Epoch 2/1000, Train Loss: 0.549599, Train-Class-Acc: {0: '56.45%', 1: '87.92%'},
Val Loss: 0.473766, Val Acc: 75.27%, Val-Class-Acc: {0: '55.60%', 1: '93.42%'}, LR: 0.000100
✅ Saved model: Class_Incremental_CL/HAR_CIL/Trained_models/EWC_CIL_v2/Period_1/HAR_MLP_epoch_2.pth
Epoch 3/1000, Train Loss: 0.443803, Train-Class-Acc: {0: '69.60%', 1:

---
### Period 2
+ ##### Total training time: 334.56 seconds
+ ##### Model: HAR_MLP_v2
+ ##### Training and saving in *'Class_Incremental_CL/HAR_CIL/Trained_models/EWC_CIL_v2/Period_2'*
+ ##### Best Epoch: 646
#### __Val Accuracy: 97.80%__
#### __Val-Class-Acc: {0: '94.09%', 1: '95.49%', 2: '100.00%'}__

In [ ]:
# ================================
# 📌 Period 2 Configuration
# ================================
period = 2

# Load dataset
X_train, y_train = period_datasets[period]["train"]
X_val, y_val     = period_datasets[period]["test"]

# Auto-select device
device = auto_select_cuda_device()

# Set model and training config
input_size    = X_train.shape[1]
hidden_size   = 128
output_size   = len(set(y_train))
dropout       = 0.2
learning_rate = 0.0001
weight_decay  = 1e-5
num_epochs    = 1000
lambda_ewc    = 1.0

# Model saving directory
model_name = "HAR_MLP"
model_saving_folder = os.path.join("Class_Incremental_CL", "HAR_CIL", "Trained_models", "EWC_CIL_v2", "Period_2")
os.makedirs(model_saving_folder, exist_ok=True)

# Load best model from Period 1 (ignore output layer shape mismatch)
prev_path = os.path.join("Class_Incremental_CL", "HAR_CIL", "Trained_models", "EWC_CIL_v2", "Period_1", f"{model_name}_best.pth")
prev_checkpoint = torch.load(prev_path, map_location=device)
model = HAR_MLP_v2(input_size, hidden_size, output_size, dropout).to(device)

# Filter mismatched keys (especially fc2) during load
state_dict = prev_checkpoint["model_state_dict"]
model_dict = model.state_dict()
filtered_dict = {k: v for k, v in state_dict.items() if k in model_dict and model_dict[k].shape == v.shape}
model.load_state_dict(filtered_dict, strict=False)

# Compute Fisher Information Matrix (based on Period 1 training data)
criterion = nn.CrossEntropyLoss()
train_loader = DataLoader(
    TensorDataset(
        torch.tensor(period_datasets[1]["train"][0], dtype=torch.float32),
        torch.tensor(period_datasets[1]["train"][1], dtype=torch.long)
    ),
    batch_size=batch_size, shuffle=True
)
fisher_dict, params_dict = EWC.compute_fisher_and_params(model, train_loader, criterion, device=device)
ewc_state = EWC(fisher_dict, params_dict)

# Optimizer (new for Period 2)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)

# Train with EWC
train_with_ewc(
    model=model,
    output_size=output_size,
    criterion=criterion,
    optimizer=optimizer,
    X_train=X_train,
    y_train=y_train,
    X_val=X_val,
    y_val=y_val,
    scheduler=None,
    num_epochs=num_epochs,
    batch_size=batch_size,
    model_saving_folder=model_saving_folder,
    model_name=model_name,
    stop_signal_file=stop_signal_file,
    ewc=ewc_state,
    lambda_ewc=lambda_ewc,
    device=device
)

# ================================
# ✅ Summary & Resource Cleanup
# ================================
unique_classes = np.unique(y_train)
num_classes = len(unique_classes)

print(f"\n✅ Training Complete. Final model architecture:")
print(model)
print(f"📊 Class Summary → unique_classes = {unique_classes}, num_classes = {num_classes}")

# Memory cleanup
del X_train, y_train, X_val, y_val, train_loader, prev_checkpoint, state_dict, model_dict, filtered_dict
del fisher_dict, params_dict, ewc_state, model
gc.collect()
torch.cuda.empty_cache()


🎯 Automatically selected GPU:
    - CUDA Device ID : 0
    - Memory Used    : 26567 MiB
    - Device Name    : NVIDIA RTX A6000

🚀 'train_with_ewc' started.
✅ Removed existing folder: Class_Incremental_CL/HAR_CIL/Trained_models/EWC_CIL_v2/Period_2

✅ Data Overview:
X_train: torch.Size([5945, 561]), y_train: torch.Size([5945])
X_val: torch.Size([2410, 561]), y_val: torch.Size([2410])


/tmp/ipykernel_3491776/2795078744.py:31: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  prev_checkpoint = torch.load(prev_path, map_location=device)


Epoch 1/1000, Train Loss: 0.313574, Train-Class-Acc: {0: '79.24%', 1: '89.88%', 2: '95.80%'},
Val Loss: 0.146322, Val Acc: 97.26%, Val-Class-Acc: {0: '90.43%', 1: '96.43%', 2: '100.00%'}, LR: 0.000100
✅ Saved model: Class_Incremental_CL/HAR_CIL/Trained_models/EWC_CIL_v2/Period_2/HAR_MLP_epoch_1.pth
Epoch 2/1000, Train Loss: 0.097342, Train-Class-Acc: {0: '96.03%', 1: '96.51%', 2: '99.91%'},
Val Loss: 0.139701, Val Acc: 96.60%, Val-Class-Acc: {0: '84.52%', 1: '98.87%', 2: '100.00%'}, LR: 0.000100
✅ Saved model: Class_Incremental_CL/HAR_CIL/Trained_models/EWC_CIL_v2/Period_2/HAR_MLP_epoch_2.pth
Epoch 3/1000, Train Loss: 0.068778, Train-Class-Acc: {0: '97.12%', 1: '97.23%', 2: '99.94%'},
Val Loss: 0.103519, Val Acc: 97.34%, Val-Class-Acc: {0: '92.46%', 1: '94.92%', 2: '100.00%'}, LR: 0.000100
✅ Saved model: Class_Incremental_CL/HAR_CIL/Trained_models/EWC_CIL_v2/Period_2/HAR_MLP_epoch_3.pth
Epoch 4/1000, Train Loss: 0.047736, Train-Class-Acc: {0: '98.29%', 1: '98.54%', 2: '99.91%'},
Val Lo

---
### Period 3
+ ##### Total training time: 387.41 seconds
+ ##### Model: HAR_MLP_v2
+ ##### Training and saving in *'Class_Incremental_CL/HAR_CIL/Trained_models/EWC_CIL_v2/Period_3'*
+ ##### Best Epoch: 949
#### __Val Accuracy: 98.37%__
#### __Val-Class-Acc: {0: '94.50%', 1: '96.05%', 2: '100.00%', 3: '100.00%'}__

In [ ]:
# ================================
# 📌 Period 3 Configuration
# ================================
period = 3

# Load dataset
X_train, y_train = period_datasets[period]["train"]
X_val, y_val     = period_datasets[period]["test"]

# Auto-select device
device = auto_select_cuda_device()

# Set model and training config
input_size    = X_train.shape[1]
hidden_size   = 128
output_size   = len(set(y_train))
dropout       = 0.2
learning_rate = 0.0001
weight_decay  = 1e-5
num_epochs    = 1000
lambda_ewc    = 1.0

# Model saving directory
model_name = "HAR_MLP"
model_saving_folder = os.path.join("Class_Incremental_CL", "HAR_CIL", "Trained_models", "EWC_CIL_v2", "Period_3")
os.makedirs(model_saving_folder, exist_ok=True)

# Load best model from Period 2 (ignore output layer shape mismatch)
prev_path = os.path.join("Class_Incremental_CL", "HAR_CIL", "Trained_models", "EWC_CIL_v2", "Period_2", f"{model_name}_best.pth")
prev_checkpoint = torch.load(prev_path, map_location=device)
model = HAR_MLP_v2(input_size, hidden_size, output_size, dropout).to(device)

# Filter mismatched keys (especially fc2) during load
state_dict = prev_checkpoint["model_state_dict"]
model_dict = model.state_dict()
filtered_dict = {k: v for k, v in state_dict.items() if k in model_dict and model_dict[k].shape == v.shape}
model.load_state_dict(filtered_dict, strict=False)

# Compute Fisher from Period 2 training data
criterion = nn.CrossEntropyLoss()
train_loader = DataLoader(
    TensorDataset(
        torch.tensor(period_datasets[2]["train"][0], dtype=torch.float32),
        torch.tensor(period_datasets[2]["train"][1], dtype=torch.long)
    ),
    batch_size=batch_size, shuffle=True
)
fisher_dict, params_dict = EWC.compute_fisher_and_params(model, train_loader, criterion, device=device)
ewc_state = EWC(fisher_dict, params_dict)

# Optimizer for this period
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)

# Train with EWC
train_with_ewc(
    model=model,
    output_size=output_size,
    criterion=criterion,
    optimizer=optimizer,
    X_train=X_train,
    y_train=y_train,
    X_val=X_val,
    y_val=y_val,
    scheduler=None,
    num_epochs=num_epochs,
    batch_size=batch_size,
    model_saving_folder=model_saving_folder,
    model_name=model_name,
    stop_signal_file=stop_signal_file,
    ewc=ewc_state,
    lambda_ewc=lambda_ewc,
    device=device
)

# ================================
# ✅ Summary & Resource Cleanup
# ================================
unique_classes = np.unique(y_train)
num_classes = len(unique_classes)

print(f"\n✅ Training Complete. Final model architecture:")
print(model)
print(f"📊 Class Summary → unique_classes = {unique_classes}, num_classes = {num_classes}")

del X_train, y_train, X_val, y_val, train_loader, prev_checkpoint, state_dict, model_dict, filtered_dict
del fisher_dict, params_dict, ewc_state, model
gc.collect()
torch.cuda.empty_cache()


🎯 Automatically selected GPU:
    - CUDA Device ID : 1
    - Memory Used    : 18 MiB
    - Device Name    : NVIDIA RTX A6000


/tmp/ipykernel_4114800/1376862386.py:31: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  prev_checkpoint = torch.load(prev_path, map_location=device)



🚀 'train_with_ewc' started.
✅ Removed existing folder: Class_Incremental_CL/HAR_CIL/Trained_models/EWC_CIL_v2/Period_3

✅ Data Overview:
X_train: torch.Size([7352, 561]), y_train: torch.Size([7352])
X_val: torch.Size([2947, 561]), y_val: torch.Size([2947])
Epoch 1/1000, Train Loss: 0.385065, Train-Class-Acc: {0: '53.73%', 1: '95.20%', 2: '98.72%', 3: '93.18%'},
Val Loss: 0.173175, Val Acc: 96.78%, Val-Class-Acc: {0: '90.84%', 1: '96.99%', 2: '100.00%', 3: '93.67%'}, LR: 0.000100
✅ Saved model: Class_Incremental_CL/HAR_CIL/Trained_models/EWC_CIL_v2/Period_3/HAR_MLP_epoch_1.pth
Epoch 2/1000, Train Loss: 0.100426, Train-Class-Acc: {0: '94.01%', 1: '98.84%', 2: '100.00%', 3: '99.08%'},
Val Loss: 0.158755, Val Acc: 94.74%, Val-Class-Acc: {0: '74.34%', 1: '99.25%', 2: '100.00%', 3: '95.34%'}, LR: 0.000100
✅ Saved model: Class_Incremental_CL/HAR_CIL/Trained_models/EWC_CIL_v2/Period_3/HAR_MLP_epoch_2.pth
Epoch 3/1000, Train Loss: 0.050937, Train-Class-Acc: {0: '97.51%', 1: '99.13%', 2: '99.97

---
### Period 4
+ ##### Total training time: 848.43 seconds
+ ##### Model: HAR_MLP_v2
+ ##### Training and saving in *'Class_Incremental_CL/HAR_CIL/Trained_models/EWC_CIL_v2/Period_4'*
+ ##### Best Epoch: 352
#### __Val Accuracy: 97.49%__
#### __Val-Class-Acc: {0: '93.69%', 1: '96.05%', 2: '99.40%', 3: '100.00%', 4: '97.45%', 5: '98.33%'}__

In [ ]:
# ================================
# 📌 Period 4 Configuration
# ================================
period = 4

# Load dataset
X_train, y_train = period_datasets[period]["train"]
X_val, y_val     = period_datasets[period]["test"]

# Auto-select device
device = auto_select_cuda_device()

# Set model and training config
input_size    = X_train.shape[1]
hidden_size   = 128
output_size   = len(set(y_train))
dropout       = 0.2
learning_rate = 0.0001
weight_decay  = 1e-5
num_epochs    = 1000
lambda_ewc    = 1.0

# Model saving directory
model_name = "HAR_MLP"
model_saving_folder = os.path.join("Class_Incremental_CL", "HAR_CIL", "Trained_models", "EWC_CIL_v2", "Period_4")
os.makedirs(model_saving_folder, exist_ok=True)

# Load best model from Period 3 (ignore output layer shape mismatch)
prev_path = os.path.join("Class_Incremental_CL", "HAR_CIL", "Trained_models", "EWC_CIL_v2", "Period_3", f"{model_name}_best.pth")
prev_checkpoint = torch.load(prev_path, map_location=device)
model = HAR_MLP_v2(input_size, hidden_size, output_size, dropout).to(device)

# Filter mismatched keys (especially fc2) during load
state_dict = prev_checkpoint["model_state_dict"]
model_dict = model.state_dict()
filtered_dict = {k: v for k, v in state_dict.items() if k in model_dict and model_dict[k].shape == v.shape}
model.load_state_dict(filtered_dict, strict=False)

# Compute Fisher from Period 3 training data
criterion = nn.CrossEntropyLoss()
train_loader = DataLoader(
    TensorDataset(
        torch.tensor(period_datasets[3]["train"][0], dtype=torch.float32),
        torch.tensor(period_datasets[3]["train"][1], dtype=torch.long)
    ),
    batch_size=batch_size, shuffle=True
)
fisher_dict, params_dict = EWC.compute_fisher_and_params(model, train_loader, criterion, device=device)
ewc_state = EWC(fisher_dict, params_dict)

# Optimizer for this period
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)

# Train with EWC
train_with_ewc(
    model=model,
    output_size=output_size,
    criterion=criterion,
    optimizer=optimizer,
    X_train=X_train,
    y_train=y_train,
    X_val=X_val,
    y_val=y_val,
    scheduler=None,
    num_epochs=num_epochs,
    batch_size=batch_size,
    model_saving_folder=model_saving_folder,
    model_name=model_name,
    stop_signal_file=stop_signal_file,
    ewc=ewc_state,
    lambda_ewc=lambda_ewc,
    device=device
)

# ================================
# ✅ Summary & Resource Cleanup
# ================================
unique_classes = np.unique(y_train)
num_classes = len(unique_classes)

print(f"\n✅ Training Complete. Final model architecture:")
print(model)
print(f"📊 Class Summary → unique_classes = {unique_classes}, num_classes = {num_classes}")

del X_train, y_train, X_val, y_val, train_loader, prev_checkpoint, state_dict, model_dict, filtered_dict
del fisher_dict, params_dict, ewc_state, model
gc.collect()
torch.cuda.empty_cache()


🎯 Automatically selected GPU:
    - CUDA Device ID : 2
    - Memory Used    : 18 MiB
    - Device Name    : NVIDIA RTX A6000


/tmp/ipykernel_4114800/2633862097.py:31: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  prev_checkpoint = torch.load(prev_path, map_location=device)



🚀 'train_with_ewc' started.
✅ Removed existing folder: Class_Incremental_CL/HAR_CIL/Trained_models/EWC_CIL_v2/Period_4

✅ Data Overview:
X_train: torch.Size([7352, 561]), y_train: torch.Size([7352])
X_val: torch.Size([2947, 561]), y_val: torch.Size([2947])
Epoch 1/1000, Train Loss: 0.751578, Train-Class-Acc: {0: '58.48%', 1: '83.92%', 2: '77.73%', 3: '95.45%', 4: '50.33%', 5: '71.50%'},
Val Loss: 0.245227, Val Acc: 93.62%, Val-Class-Acc: {0: '79.43%', 1: '99.06%', 2: '97.98%', 3: '100.00%', 4: '94.48%', 5: '89.05%'}, LR: 0.000100
✅ Saved model: Class_Incremental_CL/HAR_CIL/Trained_models/EWC_CIL_v2/Period_4/HAR_MLP_epoch_1.pth
Epoch 2/1000, Train Loss: 0.160186, Train-Class-Acc: {0: '96.89%', 1: '98.62%', 2: '95.19%', 3: '99.93%', 4: '91.71%', 5: '93.81%'},
Val Loss: 0.172514, Val Acc: 94.60%, Val-Class-Acc: {0: '93.48%', 1: '95.86%', 2: '99.19%', 3: '100.00%', 4: '90.45%', 5: '86.67%'}, LR: 0.000100
✅ Saved model: Class_Incremental_CL/HAR_CIL/Trained_models/EWC_CIL_v2/Period_4/HAR_ML

##  __Compute FWT & Model Size__

### Period 1

In [ ]:
# ================================
# 📌 Period 1 Configuration
# ================================
period = 1

# Load processed dataset for this period
X_train, y_train = period_datasets[period]["train"]
X_val, y_val     = period_datasets[period]["test"]   # use test set for validation

# Auto-select the best CUDA device
device = auto_select_cuda_device()

# Model and training hyperparameters
input_size    = X_train.shape[1]
hidden_size   = 128
output_size   = len(set(y_train))
dropout       = 0.2
learning_rate = 0.0001
weight_decay  = 1e-5
num_epochs    = 1
lambda_ewc    = 0.0  # EWC disabled for Period 1
ewc_state     = None

# Model saving path
model_name = "HAR_MLP"
model_saving_folder = os.path.join("Class_Incremental_CL", "HAR_CIL", "TT", "EWC_CIL_v2", "Period_1")
os.makedirs(model_saving_folder, exist_ok=True)

# Initialize model, criterion, optimizer
model     = HAR_MLP_v2(input_size=input_size, hidden_size=hidden_size, output_size=output_size, dropout=dropout).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)

print_model_info(model)
# Train the model
train_with_ewc(
    model=model,
    output_size=output_size,
    criterion=criterion,
    optimizer=optimizer,
    X_train=X_train,
    y_train=y_train,
    X_val=X_val,
    y_val=y_val,
    scheduler=None,
    num_epochs=num_epochs,
    batch_size=batch_size,
    model_saving_folder=model_saving_folder,
    model_name=model_name,
    stop_signal_file=stop_signal_file,
    ewc=ewc_state,
    lambda_ewc=lambda_ewc,
    device=device
)

# ================================
# ✅ Summary & Resource Cleanup
# ================================

# Unique class summary
unique_classes = np.unique(y_train)
num_classes = len(unique_classes)

print(f"\n✅ Training Complete. Final model architecture:")
print(model)
print(f"📊 Class Summary → unique_classes = {unique_classes}, num_classes = {num_classes}")

# Clean up memory to prepare for next period
del X_train, y_train, X_val, y_val, model
gc.collect()
torch.cuda.empty_cache()


🎯 Automatically selected GPU:
    - CUDA Device ID : 1
    - Memory Used    : 765 MiB
    - Device Name    : NVIDIA RTX A6000
- Total Parameters: 89218
- Model Size (float32): 0.34 MB

🚀 'train_with_ewc' started.
✅ Removed existing folder: Class_Incremental_CL/HAR_CIL/TT/EWC_CIL_v2/Period_1

✅ Data Overview:
X_train: torch.Size([2660, 561]), y_train: torch.Size([2660])
X_val: torch.Size([1023, 561]), y_val: torch.Size([1023])
Epoch 1/1, Train Loss: 0.820664, Train-Class-Acc: {0: '61.90%', 1: '43.89%'},
Val Loss: 0.523949, Val Acc: 75.37%, Val-Class-Acc: {0: '60.69%', 1: '88.91%'}, LR: 0.000100
✅ Saved model: Class_Incremental_CL/HAR_CIL/TT/EWC_CIL_v2/Period_1/HAR_MLP_epoch_1.pth

⏳ Total training time: 0.65 seconds

🏆 Best model saved as: Class_Incremental_CL/HAR_CIL/TT/EWC_CIL_v2/Period_1/HAR_MLP_best.pth (Val Accuracy: 75.37%)

📌 Final model saved as: Class_Incremental_CL/HAR_CIL/TT/EWC_CIL_v2/Period_1/HAR_MLP_final.pth

🎯 Top 5 Best Models:
Epoch 1, Train Loss: 0.820664, Train-Acc: 

### Period 2

In [ ]:
# ================================
# 📌 FWT - Period 2 (HAR EWC)
# ================================
period = 2

X_val = period_datasets[period]["test"][0]
y_val = period_datasets[period]["test"][1]

input_size = X_val.shape[1]
hidden_size = 128
dropout = 0.2
device = auto_select_cuda_device()
output_size_prev = 2  # Period 1: Sitting, Standing → label 0, 1

# === 載入 Period 1 的最佳模型 ===
prev_model_path = os.path.join(
    "Class_Incremental_CL", "HAR_CIL", "Trained_models",
    "EWC_CIL_v2", "Period_1", "HAR_MLP_best.pth"
)
checkpoint = torch.load(prev_model_path, map_location=device)

# === Previous Model ===
previous_model = HAR_MLP_v2(input_size, hidden_size, output_size_prev, dropout).to(device)
previous_model.load_state_dict(checkpoint["model_state_dict"], strict=False)
previous_model.output_size = output_size_prev
del checkpoint
gc.collect()

# === Init Model ===
init_model = HAR_MLP_v2(input_size, hidden_size, output_size_prev, dropout).to(device)
init_model.output_size = output_size_prev

# === 資料與 FWT ===
X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val, dtype=torch.long)
known_classes = [0, 1]

fwt, acc_prev, acc_init = compute_fwt_har(previous_model, init_model, X_val_tensor, y_val_tensor, known_classes)

print(f"\n### Period 2:")
print(f"- FWT (EWC HAR Period 2, old classes {known_classes}): {fwt * 100:.2f}%")
print(f"- Accuracy by previous model: {acc_prev * 100:.2f}%")
print(f"- Accuracy by init model:     {acc_init * 100:.2f}%")


🎯 Automatically selected GPU:
    - CUDA Device ID : 2
    - Memory Used    : 5118 MiB
    - Device Name    : NVIDIA RTX A6000
📋 Total samples for known classes [0, 1]: 1023

### 🔍 FWT Debug Info (HAR):
- Total evaluated samples: 1023
- Correct (PrevModel): 939 / 1023 → Acc = 0.9179
- Correct (InitModel): 491 / 1023 → Acc = 0.4800
- FWT = Acc_prev - Acc_init = 0.4379

### Period 2:
- FWT (EWC HAR Period 2, old classes [0, 1]): 43.79%
- Accuracy by previous model: 91.79%
- Accuracy by init model:     48.00%


/tmp/ipykernel_1865198/758516492.py:20: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(prev_model_path, map_location=device)
/tmp/ipykernel_1865198/37

In [ ]:
# ================================
# 📌 Period 2 Configuration
# ================================
period = 2

# Load dataset
X_train, y_train = period_datasets[period]["train"]
X_val, y_val     = period_datasets[period]["test"]

# Auto-select device
device = auto_select_cuda_device()

# Set model and training config
input_size    = X_train.shape[1]
hidden_size   = 128
output_size   = len(set(y_train))
dropout       = 0.2
learning_rate = 0.0001
weight_decay  = 1e-5
num_epochs    = 1
lambda_ewc    = 1.0

# Model saving directory
model_name = "HAR_MLP"
model_saving_folder = os.path.join("Class_Incremental_CL", "HAR_CIL", "TT", "EWC_CIL_v2", "Period_2")
os.makedirs(model_saving_folder, exist_ok=True)

# Load best model from Period 1 (ignore output layer shape mismatch)
prev_path = os.path.join("Class_Incremental_CL", "HAR_CIL", "Trained_models", "EWC_CIL_v2", "Period_1", f"{model_name}_best.pth")
prev_checkpoint = torch.load(prev_path, map_location=device)
model = HAR_MLP_v2(input_size, hidden_size, output_size, dropout).to(device)

# Filter mismatched keys (especially fc2) during load
state_dict = prev_checkpoint["model_state_dict"]
model_dict = model.state_dict()
filtered_dict = {k: v for k, v in state_dict.items() if k in model_dict and model_dict[k].shape == v.shape}
model.load_state_dict(filtered_dict, strict=False)

# Compute Fisher Information Matrix (based on Period 1 training data)
criterion = nn.CrossEntropyLoss()
train_loader = DataLoader(
    TensorDataset(
        torch.tensor(period_datasets[1]["train"][0], dtype=torch.float32),
        torch.tensor(period_datasets[1]["train"][1], dtype=torch.long)
    ),
    batch_size=batch_size, shuffle=True
)
fisher_dict, params_dict = EWC.compute_fisher_and_params(model, train_loader, criterion, device=device)
ewc_state = EWC(fisher_dict, params_dict)

# Optimizer (new for Period 2)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
print_model_info(model)
# Train with EWC
train_with_ewc(
    model=model,
    output_size=output_size,
    criterion=criterion,
    optimizer=optimizer,
    X_train=X_train,
    y_train=y_train,
    X_val=X_val,
    y_val=y_val,
    scheduler=None,
    num_epochs=num_epochs,
    batch_size=batch_size,
    model_saving_folder=model_saving_folder,
    model_name=model_name,
    stop_signal_file=stop_signal_file,
    ewc=ewc_state,
    lambda_ewc=lambda_ewc,
    device=device
)

# ================================
# ✅ Summary & Resource Cleanup
# ================================
unique_classes = np.unique(y_train)
num_classes = len(unique_classes)

print(f"\n✅ Training Complete. Final model architecture:")
print(model)
print(f"📊 Class Summary → unique_classes = {unique_classes}, num_classes = {num_classes}")

# Memory cleanup
del X_train, y_train, X_val, y_val, train_loader, prev_checkpoint, state_dict, model_dict, filtered_dict
del fisher_dict, params_dict, ewc_state, model
gc.collect()
torch.cuda.empty_cache()


🎯 Automatically selected GPU:
    - CUDA Device ID : 0
    - Memory Used    : 1018 MiB
    - Device Name    : NVIDIA RTX A6000


/tmp/ipykernel_792860/98783376.py:30: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  prev_checkpoint = torch.load(prev_path, map_location=device)


- Total Parameters: 89347
- Model Size (float32): 0.34 MB

🚀 'train_with_ewc' started.
✅ Removed existing folder: Class_Incremental_CL/HAR_CIL/TT/EWC_CIL_v2/Period_2

✅ Data Overview:
X_train: torch.Size([5945, 561]), y_train: torch.Size([5945])
X_val: torch.Size([2410, 561]), y_val: torch.Size([2410])
Epoch 1/1, Train Loss: 0.547917, Train-Class-Acc: {0: '84.84%', 1: '82.31%', 2: '74.76%'},
Val Loss: 0.223783, Val Acc: 95.77%, Val-Class-Acc: {0: '81.67%', 1: '98.31%', 2: '99.78%'}, LR: 0.000100
✅ Saved model: Class_Incremental_CL/HAR_CIL/TT/EWC_CIL_v2/Period_2/HAR_MLP_epoch_1.pth

⏳ Total training time: 0.45 seconds

🏆 Best model saved as: Class_Incremental_CL/HAR_CIL/TT/EWC_CIL_v2/Period_2/HAR_MLP_best.pth (Val Accuracy: 95.77%)

📌 Final model saved as: Class_Incremental_CL/HAR_CIL/TT/EWC_CIL_v2/Period_2/HAR_MLP_final.pth

🎯 Top 5 Best Models:
Epoch 1, Train Loss: 0.547917, Train-Acc: {0: '84.84%', 1: '82.31%', 2: '74.76%'},
Val Loss: 0.223783, Val Acc: 95.77%, Val-Acc: {0: '81.67%',

### Period 3

In [ ]:
# ================================
# 📌 FWT - Period 3 (HAR EWC)
# ================================
period = 3

X_val = period_datasets[period]["test"][0]
y_val = period_datasets[period]["test"][1]

input_size = X_val.shape[1]
hidden_size = 128
dropout = 0.2
device = auto_select_cuda_device()
output_size_prev = 3  # Period 2: Sitting, Standing, Walking → 0, 1, 2

# === 載入 Period 2 的最佳模型 ===
prev_model_path = os.path.join(
    "Class_Incremental_CL", "HAR_CIL", "Trained_models",
    "EWC_CIL_v2", "Period_2", "HAR_MLP_best.pth"
)
checkpoint = torch.load(prev_model_path, map_location=device)

previous_model = HAR_MLP_v2(input_size, hidden_size, output_size_prev, dropout).to(device)
previous_model.load_state_dict(checkpoint["model_state_dict"], strict=False)
previous_model.output_size = output_size_prev
del checkpoint
gc.collect()

# === Init Model ===
init_model = HAR_MLP_v2(input_size, hidden_size, output_size_prev, dropout).to(device)
init_model.output_size = output_size_prev

# === 資料與 FWT ===
X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val, dtype=torch.long)
known_classes = [0, 1, 2]

fwt, acc_prev, acc_init = compute_fwt_har(previous_model, init_model, X_val_tensor, y_val_tensor, known_classes)

print(f"\n### Period 3:")
print(f"- FWT (EWC HAR Period 3, old classes {known_classes}): {fwt * 100:.2f}%")
print(f"- Accuracy by previous model: {acc_prev * 100:.2f}%")
print(f"- Accuracy by init model:     {acc_init * 100:.2f}%")


🎯 Automatically selected GPU:
    - CUDA Device ID : 2
    - Memory Used    : 5118 MiB
    - Device Name    : NVIDIA RTX A6000
📋 Total samples for known classes [0, 1, 2]: 2410

### 🔍 FWT Debug Info (HAR):
- Total evaluated samples: 2410
- Correct (PrevModel): 2339 / 2410 → Acc = 0.9705
- Correct (InitModel): 622 / 2410 → Acc = 0.2581
- FWT = Acc_prev - Acc_init = 0.7124

### Period 3:
- FWT (EWC HAR Period 3, old classes [0, 1, 2]): 71.24%
- Accuracy by previous model: 97.05%
- Accuracy by init model:     25.81%


/tmp/ipykernel_1865198/3286973532.py:20: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(prev_model_path, map_location=device)
/tmp/ipykernel_1865198/3

In [ ]:
# ================================
# 📌 Period 3 Configuration
# ================================
period = 3

# Load dataset
X_train, y_train = period_datasets[period]["train"]
X_val, y_val     = period_datasets[period]["test"]

# Auto-select device
device = auto_select_cuda_device()

# Set model and training config
input_size    = X_train.shape[1]
hidden_size   = 128
output_size   = len(set(y_train))
dropout       = 0.2
learning_rate = 0.0001
weight_decay  = 1e-5
num_epochs    = 1
lambda_ewc    = 1.0

# Model saving directory
model_name = "HAR_MLP"
model_saving_folder = os.path.join("Class_Incremental_CL", "HAR_CIL", "TT", "EWC_CIL_v2", "Period_3")
os.makedirs(model_saving_folder, exist_ok=True)

# Load best model from Period 2 (ignore output layer shape mismatch)
prev_path = os.path.join("Class_Incremental_CL", "HAR_CIL", "Trained_models", "EWC_CIL_v2", "Period_2", f"{model_name}_best.pth")
prev_checkpoint = torch.load(prev_path, map_location=device)
model = HAR_MLP_v2(input_size, hidden_size, output_size, dropout).to(device)

# Filter mismatched keys (especially fc2) during load
state_dict = prev_checkpoint["model_state_dict"]
model_dict = model.state_dict()
filtered_dict = {k: v for k, v in state_dict.items() if k in model_dict and model_dict[k].shape == v.shape}
model.load_state_dict(filtered_dict, strict=False)

# Compute Fisher from Period 2 training data
criterion = nn.CrossEntropyLoss()
train_loader = DataLoader(
    TensorDataset(
        torch.tensor(period_datasets[2]["train"][0], dtype=torch.float32),
        torch.tensor(period_datasets[2]["train"][1], dtype=torch.long)
    ),
    batch_size=batch_size, shuffle=True
)
fisher_dict, params_dict = EWC.compute_fisher_and_params(model, train_loader, criterion, device=device)
ewc_state = EWC(fisher_dict, params_dict)

# Optimizer for this period
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
print_model_info(model)
# Train with EWC
train_with_ewc(
    model=model,
    output_size=output_size,
    criterion=criterion,
    optimizer=optimizer,
    X_train=X_train,
    y_train=y_train,
    X_val=X_val,
    y_val=y_val,
    scheduler=None,
    num_epochs=num_epochs,
    batch_size=batch_size,
    model_saving_folder=model_saving_folder,
    model_name=model_name,
    stop_signal_file=stop_signal_file,
    ewc=ewc_state,
    lambda_ewc=lambda_ewc,
    device=device
)

# ================================
# ✅ Summary & Resource Cleanup
# ================================
unique_classes = np.unique(y_train)
num_classes = len(unique_classes)

print(f"\n✅ Training Complete. Final model architecture:")
print(model)
print(f"📊 Class Summary → unique_classes = {unique_classes}, num_classes = {num_classes}")

del X_train, y_train, X_val, y_val, train_loader, prev_checkpoint, state_dict, model_dict, filtered_dict
del fisher_dict, params_dict, ewc_state, model
gc.collect()
torch.cuda.empty_cache()


🎯 Automatically selected GPU:
    - CUDA Device ID : 1
    - Memory Used    : 1132 MiB
    - Device Name    : NVIDIA RTX A6000


/tmp/ipykernel_792860/984836093.py:30: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  prev_checkpoint = torch.load(prev_path, map_location=device)


- Total Parameters: 89476
- Model Size (float32): 0.34 MB

🚀 'train_with_ewc' started.
✅ Removed existing folder: Class_Incremental_CL/HAR_CIL/TT/EWC_CIL_v2/Period_3

✅ Data Overview:
X_train: torch.Size([7352, 561]), y_train: torch.Size([7352])
X_val: torch.Size([2947, 561]), y_val: torch.Size([2947])
Epoch 1/1, Train Loss: 0.697774, Train-Class-Acc: {0: '65.55%', 1: '74.09%', 2: '74.12%', 3: '74.91%'},
Val Loss: 0.174004, Val Acc: 96.20%, Val-Class-Acc: {0: '92.06%', 1: '92.48%', 2: '100.00%', 3: '93.85%'}, LR: 0.000100
✅ Saved model: Class_Incremental_CL/HAR_CIL/TT/EWC_CIL_v2/Period_3/HAR_MLP_epoch_1.pth

⏳ Total training time: 0.65 seconds

🏆 Best model saved as: Class_Incremental_CL/HAR_CIL/TT/EWC_CIL_v2/Period_3/HAR_MLP_best.pth (Val Accuracy: 96.20%)

📌 Final model saved as: Class_Incremental_CL/HAR_CIL/TT/EWC_CIL_v2/Period_3/HAR_MLP_final.pth

🎯 Top 5 Best Models:
Epoch 1, Train Loss: 0.697774, Train-Acc: {0: '65.55%', 1: '74.09%', 2: '74.12%', 3: '74.91%'},
Val Loss: 0.174004,

### Period 4

In [ ]:
# ================================
# 📌 FWT - Period 4 (HAR EWC)
# ================================
period = 4

X_val = period_datasets[period]["test"][0]
y_val = period_datasets[period]["test"][1]

input_size = X_val.shape[1]
hidden_size = 128
dropout = 0.2
device = auto_select_cuda_device()
output_size_prev = 4  # Period 3: Sitting, Standing, Walking, Laying → 0, 1, 2, 3

# === 載入 Period 3 的最佳模型 ===
prev_model_path = os.path.join(
    "Class_Incremental_CL", "HAR_CIL", "Trained_models",
    "EWC_CIL_v2", "Period_3", "HAR_MLP_best.pth"
)
checkpoint = torch.load(prev_model_path, map_location=device)

previous_model = HAR_MLP_v2(input_size, hidden_size, output_size_prev, dropout).to(device)
previous_model.load_state_dict(checkpoint["model_state_dict"], strict=False)
previous_model.output_size = output_size_prev
del checkpoint
gc.collect()

# === Init Model ===
init_model = HAR_MLP_v2(input_size, hidden_size, output_size_prev, dropout).to(device)
init_model.output_size = output_size_prev

# === FWT 計算 ===
X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val, dtype=torch.long)
known_classes = [0, 1, 3]  # 同樣排除拆分後不穩定語意的類別

fwt, acc_prev, acc_init = compute_fwt_har(previous_model, init_model, X_val_tensor, y_val_tensor, known_classes)

print(f"\n### Period 4:")
print(f"- FWT (EWC HAR Period 4, old classes {known_classes}): {fwt * 100:.2f}%")
print(f"- Accuracy by previous model: {acc_prev * 100:.2f}%")
print(f"- Accuracy by init model:     {acc_init * 100:.2f}%")


🎯 Automatically selected GPU:
    - CUDA Device ID : 2
    - Memory Used    : 5118 MiB
    - Device Name    : NVIDIA RTX A6000
📋 Total samples for known classes [0, 1, 3]: 1560

### 🔍 FWT Debug Info (HAR):
- Total evaluated samples: 1560
- Correct (PrevModel): 1492 / 1560 → Acc = 0.9564
- Correct (InitModel): 465 / 1560 → Acc = 0.2981
- FWT = Acc_prev - Acc_init = 0.6583

### Period 4:
- FWT (EWC HAR Period 4, old classes [0, 1, 3]): 65.83%
- Accuracy by previous model: 95.64%
- Accuracy by init model:     29.81%


/tmp/ipykernel_1865198/3530869363.py:20: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(prev_model_path, map_location=device)
/tmp/ipykernel_1865198/3

In [ ]:
# ================================
# 📌 Period 4 Configuration
# ================================
period = 4

# Load dataset
X_train, y_train = period_datasets[period]["train"]
X_val, y_val     = period_datasets[period]["test"]

# Auto-select device
device = auto_select_cuda_device()

# Set model and training config
input_size    = X_train.shape[1]
hidden_size   = 128
output_size   = len(set(y_train))
dropout       = 0.2
learning_rate = 0.0001
weight_decay  = 1e-5
num_epochs    = 1
lambda_ewc    = 1.0

# Model saving directory
model_name = "HAR_MLP"
model_saving_folder = os.path.join("Class_Incremental_CL", "HAR_CIL", "TT", "EWC_CIL_v2", "Period_4")
os.makedirs(model_saving_folder, exist_ok=True)

# Load best model from Period 3 (ignore output layer shape mismatch)
prev_path = os.path.join("Class_Incremental_CL", "HAR_CIL", "Trained_models", "EWC_CIL_v2", "Period_3", f"{model_name}_best.pth")
prev_checkpoint = torch.load(prev_path, map_location=device)
model = HAR_MLP_v2(input_size, hidden_size, output_size, dropout).to(device)

# Filter mismatched keys (especially fc2) during load
state_dict = prev_checkpoint["model_state_dict"]
model_dict = model.state_dict()
filtered_dict = {k: v for k, v in state_dict.items() if k in model_dict and model_dict[k].shape == v.shape}
model.load_state_dict(filtered_dict, strict=False)

# Compute Fisher from Period 3 training data
criterion = nn.CrossEntropyLoss()
train_loader = DataLoader(
    TensorDataset(
        torch.tensor(period_datasets[3]["train"][0], dtype=torch.float32),
        torch.tensor(period_datasets[3]["train"][1], dtype=torch.long)
    ),
    batch_size=batch_size, shuffle=True
)
fisher_dict, params_dict = EWC.compute_fisher_and_params(model, train_loader, criterion, device=device)
ewc_state = EWC(fisher_dict, params_dict)

# Optimizer for this period
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
print_model_info(model)
# Train with EWC
train_with_ewc(
    model=model,
    output_size=output_size,
    criterion=criterion,
    optimizer=optimizer,
    X_train=X_train,
    y_train=y_train,
    X_val=X_val,
    y_val=y_val,
    scheduler=None,
    num_epochs=num_epochs,
    batch_size=batch_size,
    model_saving_folder=model_saving_folder,
    model_name=model_name,
    stop_signal_file=stop_signal_file,
    ewc=ewc_state,
    lambda_ewc=lambda_ewc,
    device=device
)

# ================================
# ✅ Summary & Resource Cleanup
# ================================
unique_classes = np.unique(y_train)
num_classes = len(unique_classes)

print(f"\n✅ Training Complete. Final model architecture:")
print(model)
print(f"📊 Class Summary → unique_classes = {unique_classes}, num_classes = {num_classes}")

del X_train, y_train, X_val, y_val, train_loader, prev_checkpoint, state_dict, model_dict, filtered_dict
del fisher_dict, params_dict, ewc_state, model
gc.collect()
torch.cuda.empty_cache()


🎯 Automatically selected GPU:
    - CUDA Device ID : 1
    - Memory Used    : 1138 MiB
    - Device Name    : NVIDIA RTX A6000


/tmp/ipykernel_792860/3431978897.py:30: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  prev_checkpoint = torch.load(prev_path, map_location=device)


- Total Parameters: 89734
- Model Size (float32): 0.34 MB

🚀 'train_with_ewc' started.
✅ Removed existing folder: Class_Incremental_CL/HAR_CIL/TT/EWC_CIL_v2/Period_4

✅ Data Overview:
X_train: torch.Size([7352, 561]), y_train: torch.Size([7352])
X_val: torch.Size([2947, 561]), y_val: torch.Size([2947])
Epoch 1/1, Train Loss: 0.750827, Train-Class-Acc: {0: '85.38%', 1: '95.85%', 2: '78.06%', 3: '83.72%', 4: '34.39%', 5: '71.70%'},
Val Loss: 0.314743, Val Acc: 92.09%, Val-Class-Acc: {0: '86.97%', 1: '97.74%', 2: '99.19%', 3: '95.34%', 4: '90.23%', 5: '80.48%'}, LR: 0.000100
✅ Saved model: Class_Incremental_CL/HAR_CIL/TT/EWC_CIL_v2/Period_4/HAR_MLP_epoch_1.pth

⏳ Total training time: 0.43 seconds

🏆 Best model saved as: Class_Incremental_CL/HAR_CIL/TT/EWC_CIL_v2/Period_4/HAR_MLP_best.pth (Val Accuracy: 92.09%)

📌 Final model saved as: Class_Incremental_CL/HAR_CIL/TT/EWC_CIL_v2/Period_4/HAR_MLP_final.pth

🎯 Top 5 Best Models:
Epoch 1, Train Loss: 0.750827, Train-Acc: {0: '85.38%', 1: '95.8

---

## 📊 Summary: 

### ✔️ HAR - EWC: Validation Summary

| Period | Training Time (s) | Validation Accuracy | Class-wise Accuracy                                                    |
|--------|-------------------|---------------------|-------------------------------------------------------------------------|
| 1      | 149.07            | **95.31%**          | {0: 95.32%, 1: 95.30%}                                                 |
| 2      | 334.56            | **97.80%**          | {0: 94.09%, 1: 95.49%, 2: 100.00%}                                     |
| 3      | 387.41            | **98.37%**          | {0: 94.50%, 1: 96.05%, 2: 100.00%, 3: 100.00%}                         |
| 4      | 848.43            | **97.49%**          | {0: 93.69%, 1: 96.05%, 2: 99.40%, 3: 100.00%, 4: 97.45%, 5: 98.33%}    |


### 🧠 Continual Learning Metrics

| Period | AA_old (%) | AA_new (%) | BWT (%) | FWT (%) | FWT Classes     | Prev. Model Acc | Init Model Acc |
|--------|------------|------------|---------|---------|------------------|------------------|-----------------|
| 2      | 94.79%     | 100.00%    | -0.33%  | 43.79%  | [0, 1]           | 91.79%           | 48.00%          |
| 3      | 96.85%     | 100.00%    | +0.32%  | 71.24%  | [0, 1, 2]        | 97.05%           | 25.81%          |
| 4      | 97.29%     | 97.89%     | -0.35%  | 65.83%  | [0, 1, 3]        | 95.64%           | 29.81%          |


### 📦 Model Size per Period

| Period | Output Size | Total Params | Δ Params vs Prev | Δ % vs Prev | Model Size (float32) |
|--------|-------------|--------------|------------------|-------------|-----------------------|
| 1      | 2           | 89,218       | —                | —           | 0.34 MB               |
| 2      | 3           | 89,347       | +129             | +0.14%      | 0.34 MB               |
| 3      | 4           | 89,476       | +129             | +0.14%      | 0.34 MB               |
| 4      | 6           | 89,734       | +258             | +0.29%      | 0.34 MB               |

**📈 Model Growth Rate (MGR) = (89,734 - 89,218) / (89,218 × 3) ≈ +0.19%**

**📈 Max trainable ratio = 100%**